In [1]:
from qbot.data.data_field import DataField
data_field = DataField(
    start_time='2024-01-01',
    end_time='2024-12-27'
)

In [ ]:
from qbot.strategy.utils import correlation_plot
correl_data, labels = correlation_plot(
    data_field['ETHUSDT']['close'],
    data_field['BTCUSDT']['close'],
    x_label='ETH',
    y_label='BTC'
)

In [ ]:
from qbot.strategy.utils import pairplot_scan
new_df = pairplot_scan(
    data_field['ETHUSDT'],
    x_col='close',
    y_col='close',
    x_shifts=range(40, 300, 70),
    y_shifts=range(-30, -300, -80),
    hue_func=lambda df: df['close_pct_change_40'] < -0.085,
)

In [ ]:
from qbot.strategy.strategy import Strategy
import numpy as np

strategy = Strategy(name='226hETHLagging')

strategy.hyper_params = {
    'shift': 226,
    'weight_scale': 25.99348249,
    'weight_sigmoid_scale': 2.23440811,
    'weight_clip_lower': -0.55174966,
    'weight_clip_upper': 0.86291262,
    'beta_offset': -0.22802245,
    'beta_scale': 1.78955622
}

def evaluate_func(strategy, data_field):

    strategy['timestamp'] = data_field['ETHUSDT']['timestamp']
    strategy['ETHUSDT_close'] = data_field['ETHUSDT']['close']
    strategy['226h_pct_change'] = strategy['ETHUSDT_close'].pct_change(periods=226)

    weight = strategy.hyper_params['weight_scale'] * strategy['226h_pct_change']
    weight = (strategy.hyper_params['weight_sigmoid_scale'] * 
                weight * (1/(1+np.exp(-weight))))
    weight = np.clip(
        weight,
        strategy.hyper_params['weight_clip_lower'],
        strategy.hyper_params['weight_clip_upper']
    )
    weight = weight - strategy.hyper_params['beta_offset']
    weight = weight * strategy.hyper_params['beta_scale']

    strategy['ETHUSDT_weight'] = weight

strategy.evaluate_func = evaluate_func
strategy.evaluate(data_field)
strategy['ETHUSDT_weight'].plot()


In [ ]:
strategy.reference_index = 'ETHUSDT_close'
asset_df = strategy.back_testing()
strategy.display_result()